In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandasql import sqldf

In [3]:
df_campaign = pd.read_csv(r'C:\Users\Saurabh Mulgaonkar\Jupyter_Notebooks\IT_Vedant_Files\Projects\Final_Project\Python_Files\cleaned_data\Campaign_data.csv')

In [4]:
df_customer = pd.read_csv(r'C:\Users\Saurabh Mulgaonkar\Jupyter_Notebooks\IT_Vedant_Files\Projects\Final_Project\Python_Files\cleaned_data\customer_master_cleaned.csv')

In [5]:
df_transaction = pd.read_csv(r'C:\Users\Saurabh Mulgaonkar\Jupyter_Notebooks\IT_Vedant_Files\Projects\Final_Project\Python_Files\cleaned_data\df_transaction_cleaned_absolute.csv')

In [6]:
#Whether there was any transaction within 1 month of campaign – accordingly create an outcome variable – 
# 0 for no transactions and 1 for transactions (the transaction should have connection to the campaign) 
#Exclude records which were not sent any campaigns
#Transactions which were not related to the campaign should not be considered either 0 or 1 but should be excluded

In [7]:
# dist of status of SMS campaign
#df_campaign[df_campaign['Campaign_Channel']=="SMS"]['status'].value_counts()

In [8]:
# customers who were sent campaigns
customers_campaigns = df_campaign['CustID'].unique()

# all customers
#all_customers = df_customer['CustID'].unique()
#df_campaign['CustID'].nunique()
#df_customer['CustID'].nunique()
#df_campaign[df_campaign['CustID']=='JFRGRFjeEJVG8RJF8R1Foj8AJJ7Et8oH']

In [9]:
# details of campaign customers only
df_transaction_req = df_transaction[df_transaction['CustID'].isin(customers_campaigns)]

In [10]:
df_campaign['Campaign_Exec_Date'] = pd.to_datetime(df_campaign['Campaign_Exec_Date'],format="%Y-%m-%d")

In [11]:
df_campaign['Campaign_End_Date'] = pd.to_datetime(df_campaign['Campaign_Exec_Date'],format="%Y-%m-%d") + pd.Timedelta(30,"days")

In [30]:
df_campaign.shape

(10912592, 6)

In [29]:
# taking data that has been viewed and/ or cliked bec the whole data set is huge
df_campaign[(df_campaign['status']=="viewed") | (df_campaign['status']=="click") | (df_campaign['status']=="clicked")]

,CustID,status,Campaign_Exec_Date,Campaign_Channel,Campaign_End_Date,date_range
0,11111J11RREFG1VoGo1FejtGEVV88JtM,viewed,2019-05-28,Email,2019-06-27,"DatetimeIndex(['2019-05-28', '2019-05-29', '20..."
14,11111J11RREFG1VoGo1FejtGEVV88JtM,viewed,2019-08-13,Email,2019-09-12,"DatetimeIndex(['2019-08-13', '2019-08-14', '20..."
18,11111J11RREFG1VoGo1FejtGEVV88JtM,viewed,2019-11-11,Email,2019-12-11,"DatetimeIndex(['2019-11-11', '2019-11-12', '20..."
19,11111J11RREFG1VoGo1FejtGEVV88JtM,viewed,2019-11-14,Email,2019-12-14,"DatetimeIndex(['2019-11-14', '2019-11-15', '20..."
20,11111J11RREFG1VoGo1FejtGEVV88JtM,viewed,2019-11-21,Email,2019-12-21,"DatetimeIndex(['2019-11-21', '2019-11-22', '20..."
...,...,...,...,...,...,...
10912585,tttttoJHGe,viewed,2020-11-22,Email,2020-12-22,0
10912586,tttttoJHGe,viewed,2020-11-23,Email,2020-12-23,0
10912587,tttttoJHGe,viewed,2020-11-24,Email,2020-12-24,0
10912588,tttttoJHGe,viewed,2020-11-25,Email,2020-12-25,0


In [12]:
df_campaign.shape

(10912592, 5)

In [13]:
def create_date_range(df,start,end):
    ser_date_range=[]
    for i in range(start,end):
        ser_date_range.append(pd.date_range(df['Campaign_Exec_Date'][i],df['Campaign_End_Date'][i]))
    return ser_date_range

In [14]:
test1 = create_date_range(df_campaign,0,10000)

In [15]:
df_campaign["date_range"] = 0

In [16]:
df_campaign["date_range"].iloc[:10000]= test1

C:\Users\Saurabh Mulgaonkar\AppData\Local\Temp\ipykernel_18740\470322135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_campaign["date_range"].iloc[:10000]= test1


In [17]:
test2 = df_campaign.iloc[:10000,:]

In [18]:
idx1 = pd.to_datetime(df_transaction_req['OrderDate'],format="%Y-%m-%d").isin(test2['date_range'][0])[pd.to_datetime(df_transaction_req['OrderDate'],format="%Y-%m-%d").isin(test2['date_range'][0])==True].index

In [20]:
df_transaction_req.loc[idx1,:].groupby('CustID').agg({'SaleValue':'sum'})

,SaleValue
CustID,
11111oGeEMotjJGVE81EjERjMHVjVtR7,2038.98
1111jHtt8FAoGj8eEtG1HRVRoG1RFeR1,38758.98
111e8VAVMVtjR8eJAJJEJJFEEJotJHAE,61196.94
1171FFVGFVE7Jj8oj8G7Fo8MGe7j11eM,48919.20
117JMJEGMMH7G1EjMFVR77jAA1eAVoM7,61198.98
...,...
ttt1jHJ71JEEG8tjGRGHeJHMeoAtFeJ8,79139.76
tttAHVetej7RAFF118RG1MRMoj88ttV8,1324.98
tttAVR1MRAFGJEjM8AE81eVMjJ7EeeHH,22647.06


In [26]:
df_campaign

,CustID,status,Campaign_Exec_Date,Campaign_Channel,Campaign_End_Date,date_range
0,11111J11RREFG1VoGo1FejtGEVV88JtM,viewed,2019-05-28,Email,2019-06-27,"DatetimeIndex(['2019-05-28', '2019-05-29', '20..."
1,11111J11RREFG1VoGo1FejtGEVV88JtM,delivered,2019-05-29,Email,2019-06-28,"DatetimeIndex(['2019-05-29', '2019-05-30', '20..."
2,11111J11RREFG1VoGo1FejtGEVV88JtM,delivered,2019-05-30,Email,2019-06-29,"DatetimeIndex(['2019-05-30', '2019-05-31', '20..."
3,11111J11RREFG1VoGo1FejtGEVV88JtM,delivered,2019-06-03,Email,2019-07-03,"DatetimeIndex(['2019-06-03', '2019-06-04', '20..."
4,11111J11RREFG1VoGo1FejtGEVV88JtM,delivered,2019-06-04,Email,2019-07-04,"DatetimeIndex(['2019-06-04', '2019-06-05', '20..."
...,...,...,...,...,...,...
10912587,tttttoJHGe,viewed,2020-11-24,Email,2020-12-24,0
10912588,tttttoJHGe,viewed,2020-11-25,Email,2020-12-25,0
10912589,tttttoJHGe,viewed,2020-11-26,Email,2020-12-26,0
10912590,tttttoJHGe,delivered,2020-11-27,Email,2020-12-27,0


In [21]:
df_campaign["date_range"].iloc[:10000] = create_date_range(df_campaign,0,10000)

In [22]:
df_campaign["date_range"].iloc[10000:30000] = create_date_range(df_campaign,10000,30000)

In [24]:
df_campaign["date_range"].iloc[30000:100000] = create_date_range(df_campaign,30000,100000)

In [ ]:
df_campaign["date_range"].iloc[100000:200000] = create_date_range(df_campaign,100000:200000)

In [ ]:
query = '''
SELECT * FROM df_transaction_req
WHERE OrderDate IN (SELECT date_range FROM test2)
GROUP BY CustID, OrderDate
'''
print(sqldf(query))

In [ ]:
query = '''
SELECT CustID, SUM(SaleValue) FROM df_transaction_req
WHERE OrderDate IN (SELECT date_range FROM test2)
GROUP BY CustID, OrderDate
'''
print(sqldf(query))

In [ ]:
campaign_dates_by_cust = df_campaign.groupby('CustID')['Campaign_Exec_Date'].apply(",".join).apply(lambda x:x.split(','))

In [ ]:
campaign_dates_by_cust

In [ ]:
x_prime = campaign_dates_by_cust.sort_index(ascending=True)[0]

In [ ]:
x = df_transaction_req[df_transaction_req['CustID']=="11111J11RREFG1VoGo1FejtGEVV88JtM"].groupby('CustID')['OrderDate'].apply(",".join).apply(lambda x: x.split(','))[0]

In [ ]:
x

In [ ]:
pd.to_datetime(x)

In [ ]:
x_prime2 = pd.date_range(pd.to_datetime(x_prime[0],format="%Y-%m-%d"),pd.to_datetime(x_prime[-1],format="%Y-%m-%d"))

In [ ]:
x_prime2

In [ ]:
any(i in pd.to_datetime(x) for i in x_prime2)

In [ ]:
df_transaction_req.groupby('CustID')[]

In [ ]:
df_transaction_req['OrderDate'].isin(campaign_dates_by_cust)

In [ ]:
df_transaction.head()

In [ ]:
# adding 30 days to exec date for campaign effectiveness consideration
#df_campaign['To_Date'] = pd.to_datetime(df_campaign['Campaign_Exec_Date'],format="%Y-%m-%d") + pd.Timedelta(30,'days')

In [ ]:
# convert to datetime
#df_campaign['Campaign_Exec_Date'] = pd.to_datetime(df_campaign['Campaign_Exec_Date'],format="%Y-%m-%d")

In [ ]:
# 11111J11RREFG1VoGo1FejtGEVV88JtM
# df_campaign['To_Date'] - pd.to_datetime(df_campaign['Campaign_Exec_Date'],format="%Y-%m-%d")
#l=[]
#for row in range(df_campaign.shape[0]):
#    date_row = list(pd.date_range(df_campaign['Campaign_Exec_Date'][row], df_campaign['To_Date'][row]).date)
#    l.append(date_row)

In [ ]:
#query = '''
#SELECT CustID, OrderDate, SUM(SaleValue) FROM df_transaction
#WHERE 
#OrderDate >= "2019-12-25" and OrderDate <= "2020-01-25"
#GROUP BY CustID, OrderDate
#'''
#print(sqldf(query))

In [ ]:
#df_transaction.groupby(['CustID']).agg({'SaleValue':'sum'})[(df_transaction['OrderDate'] >= "2019-12-25") | (df_transaction['OrderDate'] <= "2020-01-25") ]

In [ ]:
#df_transaction[df_transaction['CustID']=='JFRGRFjeEJVG8RJF8R1Foj8AJJ7Et8oH']

In [ ]:
# transaction details of customer campaigns
#df_transaction[df_transaction['CustID'].isin(customers_campaigns)]

In [ ]:
# customers found
# transaction dates such that wihtin one month there is a transaction or not